# Use case #1: using file inputParams.json

Execute the following cells only the first time running this notebook:

In [1]:
import os
os.chdir("..")

Loading the input parameters file in a new instance of the `controller`:

In [2]:
from synthDataGen.controller import DataControllerESIOS

controller = DataControllerESIOS("inputParams.json", "./synthDataGen/settings/")

Getting the first DataFrame. Then adjust it with the anual adjustments dictionary and resample it to the desired resolution:

In [3]:
df = controller.getDataFromSource()
df

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
datetime_utc,,,,,,,,,,,,,,,,
2023-07-12 18:00:00,42.36,42.36,42.36,42.36,42.36,42.36,42.36,42.36,42.36,42.36,42.36,42.36,42.36,42.36,42.36,42.36
2023-07-12 19:00:00,40.50,40.50,40.50,40.50,40.50,40.50,40.50,40.50,40.50,40.50,40.50,40.50,40.50,40.50,40.50,40.50
2023-07-12 20:00:00,41.78,41.78,41.78,41.78,41.78,41.78,41.78,41.78,41.78,41.78,41.78,41.78,41.78,41.78,41.78,41.78
2023-07-12 21:00:00,35.92,35.92,35.92,35.92,35.92,35.92,35.92,35.92,35.92,35.92,35.92,35.92,35.92,35.92,35.92,35.92
2023-07-12 22:00:00,34.75,34.75,34.75,34.75,34.75,34.75,34.75,34.75,34.75,34.75,34.75,34.75,34.75,34.75,34.75,34.75
2023-07-12 23:00:00,35.12,35.12,35.12,35.12,35.12,35.12,35.12,35.12,35.12,35.12,35.12,35.12,35.12,35.12,35.12,35.12
2023-07-13 00:00:00,29.82,29.82,29.82,29.82,29.82,29.82,29.82,29.82,29.82,29.82,29.82,29.82,29.82,29.82,29.82,29.82
2023-07-13 01:00:00,29.37,29.37,29.37,29.37,29.37,29.37,29.37,29.37,29.37,29.37,29.37,29.37,29.37,29.37,29.37,29.37
2023-07-13 02:00:00,28.44,28.44,28.44,28.44,28.44,28.44,28.44,28.44,28.44,28.44,28.44,28.44,28.44,28.44,28.44,28.44


In [4]:
df = controller.performAnualAdjustments(df)
df

[2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]


,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
datetime_utc,,,,,,,,,,,,,,,,
2023-07-12 18:00:00,423.6,423.6,423.6,423.6,423.6,423.6,423.6,423.6,423.6,423.6,423.6,423.6,423.6,423.6,423.6,423.6
2023-07-12 19:00:00,405.0,405.0,405.0,405.0,405.0,405.0,405.0,405.0,405.0,405.0,405.0,405.0,405.0,405.0,405.0,405.0
2023-07-12 20:00:00,417.8,417.8,417.8,417.8,417.8,417.8,417.8,417.8,417.8,417.8,417.8,417.8,417.8,417.8,417.8,417.8
2023-07-12 21:00:00,359.2,359.2,359.2,359.2,359.2,359.2,359.2,359.2,359.2,359.2,359.2,359.2,359.2,359.2,359.2,359.2
2023-07-12 22:00:00,347.5,347.5,347.5,347.5,347.5,347.5,347.5,347.5,347.5,347.5,347.5,347.5,347.5,347.5,347.5,347.5
2023-07-12 23:00:00,351.2,351.2,351.2,351.2,351.2,351.2,351.2,351.2,351.2,351.2,351.2,351.2,351.2,351.2,351.2,351.2
2023-07-13 00:00:00,298.2,298.2,298.2,298.2,298.2,298.2,298.2,298.2,298.2,298.2,298.2,298.2,298.2,298.2,298.2,298.2
2023-07-13 01:00:00,293.7,293.7,293.7,293.7,293.7,293.7,293.7,293.7,293.7,293.7,293.7,293.7,293.7,293.7,293.7,293.7
2023-07-13 02:00:00,284.4,284.4,284.4,284.4,284.4,284.4,284.4,284.4,284.4,284.4,284.4,284.4,284.4,284.4,284.4,284.4


In [5]:
df = controller.resampleOnAxis0(df)

import pandas as pd
pd.set_option('display.precision', 2)

df

order: 2


,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
2023-07-12 18:00:00,423.600000,423.600000,423.600000,423.600000,423.600000,423.600000,423.600000,423.600000,423.600000,423.600000,423.600000,423.600000,423.600000,423.600000,423.600000,423.600000
2023-07-12 18:15:00,414.018781,414.018781,414.018781,414.018781,414.018781,414.018781,414.018781,414.018781,414.018781,414.018781,414.018781,414.018781,414.018781,414.018781,414.018781,414.018781
2023-07-12 18:30:00,407.725041,407.725041,407.725041,407.725041,407.725041,407.725041,407.725041,407.725041,407.725041,407.725041,407.725041,407.725041,407.725041,407.725041,407.725041,407.725041
2023-07-12 18:45:00,404.718781,404.718781,404.718781,404.718781,404.718781,404.718781,404.718781,404.718781,404.718781,404.718781,404.718781,404.718781,404.718781,404.718781,404.718781,404.718781
2023-07-12 19:00:00,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000
2023-07-12 19:15:00,408.568698,408.568698,408.568698,408.568698,408.568698,408.568698,408.568698,408.568698,408.568698,408.568698,408.568698,408.568698,408.568698,408.568698,408.568698,408.568698
2023-07-12 19:30:00,415.424876,415.424876,415.424876,415.424876,415.424876,415.424876,415.424876,415.424876,415.424876,415.424876,415.424876,415.424876,415.424876,415.424876,415.424876,415.424876
2023-07-12 19:45:00,420.268616,420.268616,420.268616,420.268616,420.268616,420.268616,420.268616,420.268616,420.268616,420.268616,420.268616,420.268616,420.268616,420.268616,420.268616,420.268616
2023-07-12 20:00:00,417.800000,417.800000,417.800000,417.800000,417.800000,417.800000,417.800000,417.800000,417.800000,417.800000,417.800000,417.800000,417.800000,417.800000,417.800000,417.800000
2023-07-12 20:15:00,408.019029,408.019029,408.019029,408.019029,408.019029,408.019029,408.019029,408.019029,408.019029,408.019029,408.019029,408.019029,408.019029,408.019029,408.019029,408.019029
